In [1]:
import torch
from torch import nn, optim
from opacus import PrivacyEngine
from sklearn.metrics import accuracy_score

# Simulated dataset
X = torch.randn(500, 128)
y = torch.randint(0, 6, (500,))
dataset = torch.utils.data.TensorDataset(X, y)

# Initial settings
batch_size = 32
noise_multiplier = 1.0
max_grad_norm = 1.0
target_epsilon = 8  # Stop if we reach this privacy budget
delta = 1e-5

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Simple model
class EmotionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 6)
        )
    def forward(self, x):
        return self.fc(x)

model = EmotionNet()
optimizer = optim.SGD(model.parameters(), lr=0.05)
criterion = nn.CrossEntropyLoss()

# Add DP
privacy_engine = PrivacyEngine()
model, optimizer, dataloader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=dataloader,
    noise_multiplier=noise_multiplier,
    max_grad_norm=max_grad_norm,
)

# Training loop with adaptive DP
for epoch in range(20):  # Max 20 epochs
    model.train()
    epoch_losses = []
    all_preds, all_labels = [], []

    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
        all_preds.extend(preds.argmax(1).detach().cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

    # Compute accuracy & loss
    epoch_loss = sum(epoch_losses) / len(epoch_losses)
    epoch_acc = accuracy_score(all_labels, all_preds)

    # Get current privacy budget
    epsilon = privacy_engine.get_epsilon(delta)
    print(f"Epoch {epoch+1}: Loss={epoch_loss:.4f}, Acc={epoch_acc:.2f}, ε={epsilon:.2f}, noise={noise_multiplier}, batch={batch_size}")

    # Stop if privacy budget is reached
    if epsilon > target_epsilon:
        print("Reached target privacy budget. Stopping training.")
        break

    # Adaptive adjustments:
    if epsilon > target_epsilon * 0.8:  # If close to budget, increase noise
        noise_multiplier += 0.2
        print(f"Increasing noise to {noise_multiplier:.2f} to slow budget growth.")

    if epsilon > target_epsilon * 0.9:  # If very close, also reduce batch size
        batch_size = max(8, batch_size // 2)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        print(f"Reducing batch size to {batch_size} to preserve privacy.")

# Save trained model
torch.save(model.state_dict(), "dp_emotion_model.pth")
print("Model trained & saved with DP.")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


Epoch 1: Loss=1.8107, Acc=0.15, ε=2.25, noise=1.0, batch=32
Epoch 2: Loss=1.7784, Acc=0.20, ε=2.81, noise=1.0, batch=32
Epoch 3: Loss=1.7724, Acc=0.22, ε=3.25, noise=1.0, batch=32
Epoch 4: Loss=1.7747, Acc=0.21, ε=3.64, noise=1.0, batch=32
Epoch 5: Loss=1.7564, Acc=0.25, ε=3.99, noise=1.0, batch=32
Epoch 6: Loss=1.7765, Acc=0.23, ε=4.31, noise=1.0, batch=32
Epoch 7: Loss=1.7383, Acc=0.28, ε=4.61, noise=1.0, batch=32
Epoch 8: Loss=1.7377, Acc=0.27, ε=4.90, noise=1.0, batch=32
Epoch 9: Loss=1.7073, Acc=0.30, ε=5.17, noise=1.0, batch=32
Epoch 10: Loss=1.7331, Acc=0.26, ε=5.43, noise=1.0, batch=32
Epoch 11: Loss=1.7154, Acc=0.28, ε=5.68, noise=1.0, batch=32
Epoch 12: Loss=1.7129, Acc=0.25, ε=5.92, noise=1.0, batch=32
Epoch 13: Loss=1.7182, Acc=0.27, ε=6.15, noise=1.0, batch=32
Epoch 14: Loss=1.7152, Acc=0.31, ε=6.38, noise=1.0, batch=32
Epoch 15: Loss=1.6968, Acc=0.30, ε=6.60, noise=1.0, batch=32
Increasing noise to 1.20 to slow budget growth.
Epoch 16: Loss=1.6693, Acc=0.28, ε=6.82, noise